# Join the Feature tables

The goal in this step is to:
- Load the datasets:
- Merge the datasets based on the foreign key
- Perform Basic QA
- Export the dataset


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Set up PATH variable
CLEAN_DIR = Path("../data/clean")
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

app_train_fe = pd.read_csv(CLEAN_DIR / "application_train_fe.csv")
app_test_fe  = pd.read_csv(CLEAN_DIR / "application_test_fe.csv")
bureau_agg   = pd.read_csv(CLEAN_DIR / "bureau_agg.csv")
behavior_agg = pd.read_csv(CLEAN_DIR / "behavior_agg.csv")

print("Loaded:")
print("app_train_fe:", app_train_fe.shape)
print("app_test_fe:", app_test_fe.shape)
print("bureau_agg:", bureau_agg.shape)
print("behavior_agg:", behavior_agg.shape)


Loaded:
app_train_fe: (307511, 137)
app_test_fe: (48744, 136)
bureau_agg: (305811, 22)
behavior_agg: (338857, 24)


### Merge the train dataset

In [2]:
train_merged = (
    app_train_fe
    .merge(bureau_agg, on="SK_ID_CURR", how="left")
    .merge(behavior_agg, on="SK_ID_CURR", how="left")
)
print("train_merged shape:", train_merged.shape)


train_merged shape: (307511, 181)


### Merge the test set

In [3]:
test_merged = (
    app_test_fe
    .merge(bureau_agg, on="SK_ID_CURR", how="left")
    .merge(behavior_agg, on="SK_ID_CURR", how="left")
)
print("test_merged shape:", test_merged.shape)


test_merged shape: (48744, 180)


### Quality check - Missing values

In [4]:
def missing_summary(df, top_n=30):
    mis = df.isnull().mean().sort_values(ascending=False)
    return (
        mis.to_frame("missing_frac")
        .assign(missing_pct=lambda x: x["missing_frac"]*100)
        .head(top_n)
    )

print("Merged train missing top 30:")
display(missing_summary(train_merged, 30))


Merged train missing top 30:


,missing_frac,missing_pct
CC_MAX_UTILIZATION,0.749404,74.940409
CC_MEAN_UTILIZATION,0.749404,74.940409
CC_MEAN_LIMIT,0.746565,74.656516
CC_MEAN_BALANCE,0.746565,74.656516
CC_MEAN_NUM_MONTHS,0.746565,74.656516
CC_NUM_PREV,0.746565,74.656516
BUREAU_AVG_NUM_MONTHS_REPORTED,0.700073,70.007252
BUREAU_MEAN_BAD_MONTH_RATE,0.700073,70.007252
BUREAU_MAX_BAD_MONTH_RATE,0.700073,70.007252
COMMONAREA_AVG,0.698723,69.872297


### Quality check: Target presence and distribution again

In [5]:
if "TARGET" in train_merged.columns:
    print("TARGET mean:", train_merged["TARGET"].mean())
    print(train_merged["TARGET"].value_counts())


TARGET mean: 0.08072881945686496
TARGET
0    282686
1     24825
Name: count, dtype: int64


### Quality check: Duplicates

In [6]:
dup_train = train_merged["SK_ID_CURR"].duplicated().sum()
dup_test = test_merged["SK_ID_CURR"].duplicated().sum()
print("Duplicate clients in train_merged:", dup_train)
print("Duplicate clients in test_merged:", dup_test)


Duplicate clients in train_merged: 0
Duplicate clients in test_merged: 0


### Export merged dataset

In [7]:
train_merge_path = CLEAN_DIR / "train_merged.csv"
test_merge_path  = CLEAN_DIR / "test_merged.csv"

train_merged.to_csv(train_merge_path, index=False)
test_merged.to_csv(test_merge_path, index=False)

print(f"Saved {train_merge_path}")
print(f"Saved {test_merge_path}")


Saved ../data/clean/train_merged.csv
Saved ../data/clean/test_merged.csv
